In [1]:
import pandas as pd
from tqdm import tqdm
import subprocess

In [2]:
# Function for executing os commands
def terminal(cmd):
    stdout, stderr = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    if stderr != b'':
        return "error"
    return stdout.decode('utf-8')

In [3]:
# Check if folder exists
if terminal("cd csse_covid_19_daily_reports") == "error": 
    dataExists = False
else: 
    dataExists = True

In [4]:
# Install Subversion: brew install subversion

# Download data -> old data is removed if it exists
if dataExists: 
    terminal("rm -r csse_covid_19_daily_reports")
status = terminal("svn export https://github.com/CSSEGISandData/COVID-19/trunk/csse_covid_19_data/csse_covid_19_daily_reports")

if status != "error": 
    print("International Data downloaded")
else: 
    print("Error!")

International Data downloaded


In [5]:
# Install wget: brew install wget

# Download data -> old data is removed if it exists
terminal("rm COVID19_Fallzahlen_CH_total_v2.csv")
terminal("wget https://github.com/openZH/covid_19/raw/master/COVID19_Fallzahlen_CH_total_v2.csv")

print("Local Data downloaded - check manually!")

Local Data downloaded - check manually!


In [6]:
# Get all filenames from the international data

files = terminal("cd csse_covid_19_daily_reports; ls *.csv;").split('\n')[:-1]
files = ["csse_covid_19_daily_reports/" + file for file in files]

In [7]:
# Join all the files into a big dataframe
data = pd.DataFrame()
for filename in tqdm(files):
    temp = pd.read_csv(filename)
    temp["Date"] = filename.split('/')[1].split('.')[0]
    data = data.append(temp)

100%|█████████████████████████████████████████| 811/811 [01:27<00:00,  9.26it/s]


In [8]:
# Inspect data
data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,...,Incident_Rate,Case_Fatality_Ratio,Date,Province/State,Country/Region,Last Update,Latitude,Longitude,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-02 05:22:33,33.939110,67.709953,52513.0,2201.0,41727.0,...,134.896578,4.191343,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Albania,2021-01-02 05:22:33,41.153300,20.168300,58316.0,1181.0,33634.0,...,2026.409062,2.025173,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Algeria,2021-01-02 05:22:33,28.033900,1.659600,99897.0,2762.0,67395.0,...,227.809861,2.764848,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Andorra,2021-01-02 05:22:33,42.506300,1.521800,8117.0,84.0,7463.0,...,10505.403482,1.034865,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Angola,2021-01-02 05:22:33,-11.202700,17.873900,17568.0,405.0,11146.0,...,53.452981,2.305328,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,NaN,NaN,NaN,Zimbabwe,2022-01-01 04:22:14,-19.015438,29.154857,213258.0,5004.0,NaN,...,1434.831780,2.346454,12-31-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4006,NaN,NaN,NaN,Winter Olympics 2022,2022-01-01 04:22:14,39.904200,116.407400,0.0,0.0,0.0,...,0.000000,0.000000,12-31-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4007,NaN,NaN,NaN,Antarctica,2022-01-01 04:22:14,-71.949900,23.347000,11.0,0.0,0.0,...,0.000000,0.000000,12-31-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4008,NaN,NaN,Jersey,United Kingdom,2022-01-01 04:22:14,49.213800,-2.135800,19002.0,89.0,0.0,...,0.000000,0.000000,12-31-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Set Dates to correct format
data["Date"] = pd.to_datetime(data["Date"])
data["Last_Update"] = pd.to_datetime(data["Last_Update"])

In [10]:
# Inspect suspect values
data.loc[pd.notnull(data["Longitude"])] 

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,...,Incident_Rate,Case_Fatality_Ratio,Date,Province/State,Country/Region,Last Update,Latitude,Longitude,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,66907.0,2761.0,31536.0,...,NaN,NaN,2020-03-01,Hubei,Mainland China,2020-03-01T10:13:19,30.975600,112.27070,NaN,NaN
1,NaN,NaN,NaN,NaN,NaT,NaN,NaN,3736.0,17.0,30.0,...,NaN,NaN,2020-03-01,NaN,South Korea,2020-03-01T23:43:03,36.000000,128.00000,NaN,NaN
2,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1694.0,34.0,83.0,...,NaN,NaN,2020-03-01,NaN,Italy,2020-03-01T23:23:02,43.000000,12.00000,NaN,NaN
3,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1349.0,7.0,1016.0,...,NaN,NaN,2020-03-01,Guangdong,Mainland China,2020-03-01T14:13:18,23.341700,113.42440,NaN,NaN
4,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1272.0,22.0,1198.0,...,NaN,NaN,2020-03-01,Henan,Mainland China,2020-03-01T14:13:18,33.882000,113.61400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,NaN,Tonga,2020-03-21T10:13:08,-21.179000,-175.19820,NaN,NaN
326,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,NaN,Winter Olympics 2022,2020-03-21T10:13:08,39.904200,116.40740,NaN,NaN
327,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,NaN,Antarctica,2020-03-21T10:13:08,-71.949900,23.34700,NaN,NaN
328,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,Jersey,United Kingdom,2020-03-21T10:13:08,49.213800,-2.13580,NaN,NaN


In [11]:
# Every value in the correctionArray has two values, the first one is the wrong column, the second one the correct
correctionArray = [["Latitude", "Lat"],
                   ["Longitude", "Long_"],
                   ["Case-Fatality_Ratio", "Case_Fatality_Ratio"],
                   ["Last Update", "Last_Update"],
                   ["Country/Region", "Country_Region"],
                   ["Province/State", "Province_State"],
                   ["Incidence_Rate", "Incident_Rate"]]

for correction in tqdm(correctionArray):
    data.loc[pd.notnull(data[correction[0]]), correction[1]] = data.loc[pd.notnull(data[correction[0]]), correction[0]]

100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 10.87it/s]


In [12]:
# Inspect suspect values again
data.loc[pd.notnull(data["Longitude"])] 

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,...,Incident_Rate,Case_Fatality_Ratio,Date,Province/State,Country/Region,Last Update,Latitude,Longitude,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,Hubei,Mainland China,2020-03-01 10:13:19,30.975600,112.27070,66907.0,2761.0,31536.0,...,NaN,NaN,2020-03-01,Hubei,Mainland China,2020-03-01T10:13:19,30.975600,112.27070,NaN,NaN
1,NaN,NaN,NaN,South Korea,2020-03-01 23:43:03,36.000000,128.00000,3736.0,17.0,30.0,...,NaN,NaN,2020-03-01,NaN,South Korea,2020-03-01T23:43:03,36.000000,128.00000,NaN,NaN
2,NaN,NaN,NaN,Italy,2020-03-01 23:23:02,43.000000,12.00000,1694.0,34.0,83.0,...,NaN,NaN,2020-03-01,NaN,Italy,2020-03-01T23:23:02,43.000000,12.00000,NaN,NaN
3,NaN,NaN,Guangdong,Mainland China,2020-03-01 14:13:18,23.341700,113.42440,1349.0,7.0,1016.0,...,NaN,NaN,2020-03-01,Guangdong,Mainland China,2020-03-01T14:13:18,23.341700,113.42440,NaN,NaN
4,NaN,NaN,Henan,Mainland China,2020-03-01 14:13:18,33.882000,113.61400,1272.0,22.0,1198.0,...,NaN,NaN,2020-03-01,Henan,Mainland China,2020-03-01T14:13:18,33.882000,113.61400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,NaN,NaN,NaN,Tonga,2020-03-21 10:13:08,-21.179000,-175.19820,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,NaN,Tonga,2020-03-21T10:13:08,-21.179000,-175.19820,NaN,NaN
326,NaN,NaN,NaN,Winter Olympics 2022,2020-03-21 10:13:08,39.904200,116.40740,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,NaN,Winter Olympics 2022,2020-03-21T10:13:08,39.904200,116.40740,NaN,NaN
327,NaN,NaN,NaN,Antarctica,2020-03-21 10:13:08,-71.949900,23.34700,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,NaN,Antarctica,2020-03-21T10:13:08,-71.949900,23.34700,NaN,NaN
328,NaN,NaN,Jersey,United Kingdom,2020-03-21 10:13:08,49.213800,-2.13580,0.0,0.0,0.0,...,NaN,NaN,2020-03-21,Jersey,United Kingdom,2020-03-21T10:13:08,49.213800,-2.13580,NaN,NaN


In [13]:
# delete no longer needed columns
for correction in tqdm(correctionArray):
    data = data.drop(columns = correction[0])

100%|█████████████████████████████████████████████| 7/7 [00:01<00:00,  4.69it/s]


In [14]:
# Inspect data again
data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Date
0,NaN,NaN,NaN,Afghanistan,2021-01-02 05:22:33,33.939110,67.709953,52513.0,2201.0,41727.0,8585.0,Afghanistan,134.896578,4.191343,2021-01-01
1,NaN,NaN,NaN,Albania,2021-01-02 05:22:33,41.153300,20.168300,58316.0,1181.0,33634.0,23501.0,Albania,2026.409062,2.025173,2021-01-01
2,NaN,NaN,NaN,Algeria,2021-01-02 05:22:33,28.033900,1.659600,99897.0,2762.0,67395.0,29740.0,Algeria,227.809861,2.764848,2021-01-01
3,NaN,NaN,NaN,Andorra,2021-01-02 05:22:33,42.506300,1.521800,8117.0,84.0,7463.0,570.0,Andorra,10505.403482,1.034865,2021-01-01
4,NaN,NaN,NaN,Angola,2021-01-02 05:22:33,-11.202700,17.873900,17568.0,405.0,11146.0,6017.0,Angola,53.452981,2.305328,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,NaN,NaN,NaN,Zimbabwe,2022-01-01 04:22:14,-19.015438,29.154857,213258.0,5004.0,NaN,NaN,Zimbabwe,1434.831780,2.346454,2021-12-31
4006,NaN,NaN,NaN,Winter Olympics 2022,2022-01-01 04:22:14,39.904200,116.407400,0.0,0.0,0.0,0.0,Winter Olympics 2022,0.000000,0.000000,2021-12-31
4007,NaN,NaN,NaN,Antarctica,2022-01-01 04:22:14,-71.949900,23.347000,11.0,0.0,0.0,0.0,Antarctica,0.000000,0.000000,2021-12-31
4008,NaN,NaN,Jersey,United Kingdom,2022-01-01 04:22:14,49.213800,-2.135800,19002.0,89.0,0.0,0.0,"Jersey, United Kingdom",0.000000,0.000000,2021-12-31


In [15]:
# Rename columns
data.rename(columns = {
    "Province_State":"Province or State",
    "Country_Region":"Country or Region",
    "Last_Update":"Last Update",
    "Lat":"Latitude",
    "Long_":"Longitude",
    "Incident_Rate":"Incident Rate",
    "Case_Fatality_Ratio":"Case Fatality Ratio"
}, inplace = True)

data

,FIPS,Admin2,Province or State,Country or Region,Last Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident Rate,Case Fatality Ratio,Date
0,NaN,NaN,NaN,Afghanistan,2021-01-02 05:22:33,33.939110,67.709953,52513.0,2201.0,41727.0,8585.0,Afghanistan,134.896578,4.191343,2021-01-01
1,NaN,NaN,NaN,Albania,2021-01-02 05:22:33,41.153300,20.168300,58316.0,1181.0,33634.0,23501.0,Albania,2026.409062,2.025173,2021-01-01
2,NaN,NaN,NaN,Algeria,2021-01-02 05:22:33,28.033900,1.659600,99897.0,2762.0,67395.0,29740.0,Algeria,227.809861,2.764848,2021-01-01
3,NaN,NaN,NaN,Andorra,2021-01-02 05:22:33,42.506300,1.521800,8117.0,84.0,7463.0,570.0,Andorra,10505.403482,1.034865,2021-01-01
4,NaN,NaN,NaN,Angola,2021-01-02 05:22:33,-11.202700,17.873900,17568.0,405.0,11146.0,6017.0,Angola,53.452981,2.305328,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,NaN,NaN,NaN,Zimbabwe,2022-01-01 04:22:14,-19.015438,29.154857,213258.0,5004.0,NaN,NaN,Zimbabwe,1434.831780,2.346454,2021-12-31
4006,NaN,NaN,NaN,Winter Olympics 2022,2022-01-01 04:22:14,39.904200,116.407400,0.0,0.0,0.0,0.0,Winter Olympics 2022,0.000000,0.000000,2021-12-31
4007,NaN,NaN,NaN,Antarctica,2022-01-01 04:22:14,-71.949900,23.347000,11.0,0.0,0.0,0.0,Antarctica,0.000000,0.000000,2021-12-31
4008,NaN,NaN,Jersey,United Kingdom,2022-01-01 04:22:14,49.213800,-2.135800,19002.0,89.0,0.0,0.0,"Jersey, United Kingdom",0.000000,0.000000,2021-12-31


In [16]:
# Remove redundant data
data = data.drop(columns = "Combined_Key")

In [17]:
# Index Data
data = data.sort_values(["Date", "Country or Region", "Province or State"])
data = data.set_index(["Date", "Country or Region", "Province or State"])
data

FIPS Admin2  \
Date       Country or Region    Province or State                
2020-01-22 Antarctica           NaN                 NaN    NaN   
           China                Unknown             NaN    NaN   
           Hong Kong            Hong Kong           NaN    NaN   
           Japan                NaN                 NaN    NaN   
           Kiribati             NaN                 NaN    NaN   
...                                                 ...    ...   
2022-04-11 West Bank and Gaza   NaN                 NaN    NaN   
           Winter Olympics 2022 NaN                 NaN    NaN   
           Yemen                NaN                 NaN    NaN   
           Zambia               NaN                 NaN    NaN   
           Zimbabwe             NaN                 NaN    NaN   

                                                          Last Update  \
Date       Country or Region    Province or State                       
2020-01-22 Antarctica           NaN               2020-01-22 17:00:00   
           China                Unknown           2020-01-22 17:00:00   
           Hong Kong            Hong Kong         2020-01-22 17:00:00   
           Japan                NaN               2020-01-22 17:00:00   
           Kiribati             NaN               2020-01-22 17:00:00   
...                                                               ...   
2022-04-11 West Bank and Gaza   NaN               2022-04-12 04:20:49   
           Winter Olympics 2022 NaN               2022-04-12 04:20:49   
           Yemen                NaN               2022-04-12 04:20:49   
           Zambia               NaN               2022-04-12 04:20:49   
           Zimbabwe             NaN               2022-04-12 04:20:49   

                                                    Latitude   Longitude  \
Date       Country or Region    Province or State                          
2020-01-22 Antarctica           NaN                      NaN         NaN   
           China                Unknown                  NaN         NaN   
           Hong Kong            Hong Kong                NaN         NaN   
           Japan                NaN                      NaN         NaN   
           Kiribati             NaN                      NaN         NaN   
...                                                      ...         ...   
2022-04-11 West Bank and Gaza   NaN                31.952200   35.233200   
           Winter Olympics 2022 NaN                39.904200  116.407400   
           Yemen                NaN                15.552727   48.516388   
           Zambia               NaN               -13.133897   27.849332   
           Zimbabwe             NaN               -19.015438   29.154857   

                                                   Confirmed  Deaths  \
Date       Country or Region    Province or State                      
2020-01-22 Antarctica           NaN                      0.0     0.0   
           China                Unknown                  NaN     NaN   
           Hong Kong            Hong Kong                NaN     NaN   
           Japan                NaN                      2.0     NaN   
           Kiribati             NaN                      0.0     0.0   
...                                                      ...     ...   
2022-04-11 West Bank and Gaza   NaN                 656287.0  5655.0   
           Winter Olympics 2022 NaN                    535.0     0.0   
           Yemen                NaN                  11814.0  2147.0   
           Zambia               NaN                 317804.0  3967.0   
           Zimbabwe             NaN                 247010.0  5460.0   

                                                   Recovered  Active  \
Date       Country or Region    Province or State                      
2020-01-22 Antarctica           NaN                      0.0     NaN   
           China                Unknown                  NaN     NaN   
           Hong Kong        

In [18]:
# Check last unknown columns
data.loc[pd.notnull(data["FIPS"])] # Fips probably is the postal code and Admin2 the area

FIPS      Admin2  \
Date       Country or Region Province or State                        
2020-03-22 US                Alabama             1073.0   Jefferson   
                             Alabama             1117.0      Shelby   
                             Alabama             1081.0         Lee   
                             Alabama             1089.0     Madison   
                             Alabama             1125.0  Tuscaloosa   
...                                                 ...         ...   
2022-04-11 US                Wyoming            56039.0       Teton   
                             Wyoming            56041.0       Uinta   
                             Wyoming            90056.0  Unassigned   
                             Wyoming            56043.0    Washakie   
                             Wyoming            56045.0      Weston   

                                                       Last Update   Latitude  \
Date       Country or Region Province or State                                  
2020-03-22 US                Alabama           2020-03-22 23:45:00  33.555547   
                             Alabama           2020-03-22 23:45:00  33.268798   
                             Alabama           2020-03-22 23:45:00  32.601549   
                             Alabama           2020-03-22 23:45:00  34.763271   
                             Alabama           2020-03-22 23:45:00  33.287261   
...                                                            ...        ...   
2022-04-11 US                Wyoming           2022-04-12 04:20:49  43.935225   
                             Wyoming           2022-04-12 04:20:49  41.287818   
                             Wyoming           2022-04-12 04:20:49        NaN   
                             Wyoming           2022-04-12 04:20:49  43.904516   
                             Wyoming           2022-04-12 04:20:49  43.839612   

                                                 Longitude  Confirmed  Deaths  \
Date       Country or Region Province or State                                  
2020-03-22 US                Alabama            -86.895063       78.0     0.0   
                             Alabama            -86.662326       21.0     0.0   
                             Alabama            -85.351322       16.0     0.0   
                             Alabama            -86.550696       22.0     0.0   
                             Alabama            -87.525568        9.0     0.0   
...                                                    ...        ...     ...   
2022-04-11 US                Wyoming           -110.589080     9932.0    16.0   
                             Wyoming           -110.547578     5634.0    39.0   
                             Wyoming                   NaN        0.0     0.0   
                             Wyoming           -107.680187     2358.0    43.0   
                             Wyoming           -104.567488     1588.0    18.0   

                                                Recovered  Active  \
Date       Country or Region Province or State                      
2020-03-22 US                Alabama                  0.0    78.0   
                             Alabama                  0.0    21.0   
                             Alabama                  0.0    16.0   
                             Alabama                  0.0    22.0   
                             Alabama                  0.0     9.0   
...                                                   ...     ...   
2022-04-11 US                Wyoming                  NaN     NaN   
                             Wyoming                  NaN     NaN   
                             Wyoming                  NaN     NaN   
                             Wyoming                  NaN     NaN   
                             Wyoming                  NaN     NaN   

                                                Incident Rate  \
Date       Country or Region Province or State          

In [19]:
# Another data cleanup
data = data.drop(columns = "FIPS")
data.rename(columns = {
    "Admin2":"Area"
}, inplace = True)

data = data.reset_index(drop = False)
data = data.sort_values(["Date", "Country or Region", "Province or State", "Area"])
data = data.set_index(["Date", "Country or Region", "Province or State", "Area"])
data

Last Update  \
Date       Country or Region    Province or State Area                       
2020-01-22 Antarctica           NaN               NaN  2020-01-22 17:00:00   
           China                Unknown           NaN  2020-01-22 17:00:00   
           Hong Kong            Hong Kong         NaN  2020-01-22 17:00:00   
           Japan                NaN               NaN  2020-01-22 17:00:00   
           Kiribati             NaN               NaN  2020-01-22 17:00:00   
...                                                                    ...   
2022-04-11 West Bank and Gaza   NaN               NaN  2022-04-12 04:20:49   
           Winter Olympics 2022 NaN               NaN  2022-04-12 04:20:49   
           Yemen                NaN               NaN  2022-04-12 04:20:49   
           Zambia               NaN               NaN  2022-04-12 04:20:49   
           Zimbabwe             NaN               NaN  2022-04-12 04:20:49   

                                                         Latitude   Longitude  \
Date       Country or Region    Province or State Area                          
2020-01-22 Antarctica           NaN               NaN         NaN         NaN   
           China                Unknown           NaN         NaN         NaN   
           Hong Kong            Hong Kong         NaN         NaN         NaN   
           Japan                NaN               NaN         NaN         NaN   
           Kiribati             NaN               NaN         NaN         NaN   
...                                                           ...         ...   
2022-04-11 West Bank and Gaza   NaN               NaN   31.952200   35.233200   
           Winter Olympics 2022 NaN               NaN   39.904200  116.407400   
           Yemen                NaN               NaN   15.552727   48.516388   
           Zambia               NaN               NaN  -13.133897   27.849332   
           Zimbabwe             NaN               NaN  -19.015438   29.154857   

                                                        Confirmed  Deaths  \
Date       Country or Region    Province or State Area                      
2020-01-22 Antarctica           NaN               NaN         0.0     0.0   
           China                Unknown           NaN         NaN     NaN   
           Hong Kong            Hong Kong         NaN         NaN     NaN   
           Japan                NaN               NaN         2.0     NaN   
           Kiribati             NaN               NaN         0.0     0.0   
...                                                           ...     ...   
2022-04-11 West Bank and Gaza   NaN               NaN    656287.0  5655.0   
           Winter Olympics 2022 NaN               NaN       535.0     0.0   
           Yemen                NaN               NaN     11814.0  2147.0   
           Zambia               NaN               NaN    317804.0  3967.0   
           Zimbabwe             NaN               NaN    247010.0  5460.0   

                                                        Recovered  Active  \
Date       Country or Region    Province or State Area                      
2020-01-22 Antarctica           NaN               NaN         0.0     NaN   
           China                Unknown           NaN         NaN     NaN   
           Hong Kong            Hong Kong         NaN         NaN     NaN   
           Japan                NaN               NaN         NaN     NaN   
           Kiribati             NaN               NaN         0.0     NaN   
...                                                           ...     ...   
2022-04-11 West Bank and Gaza   NaN               NaN         NaN     NaN   
           Winter Olympics 2022 NaN               NaN         NaN     NaN   
           Yemen                NaN               NaN         NaN     NaN   
           Zambia               NaN               NaN         NaN     NaN   
           Zimbabwe             NaN               NaN         NaN   

In [20]:
# Read in local CH data
data_local = pd.read_csv("COVID19_Fallzahlen_CH_total_v2.csv")
data_local

,date,time,abbreviation_canton_and_fl,ncumul_tested,ncumul_conf,new_hosp,current_hosp,current_icu,current_vent,ncumul_released,ncumul_deceased,source,current_isolated,current_quarantined,ncumul_confirmed_non_resident,current_hosp_non_resident,current_quarantined_riskareatravel,current_quarantined_total,current_hosp_resident
0,2020-01-05,00:00,GE,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.0,https://infocovid.smc.unige.ch/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-30,00:00,GE,NaN,5.0,NaN,NaN,NaN,NaN,NaN,0.0,https://infocovid.smc.unige.ch/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-01,00:00,GE,109.0,5.0,NaN,NaN,NaN,NaN,NaN,0.0,https://infocovid.smc.unige.ch/,1.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2020-02-02,00:00,GE,115.0,5.0,NaN,NaN,NaN,NaN,NaN,0.0,https://infocovid.smc.unige.ch/,1.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2020-02-03,00:00,GE,136.0,5.0,NaN,NaN,NaN,NaN,NaN,0.0,https://infocovid.smc.unige.ch/,1.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18833,2022-04-12,NaN,LU,NaN,NaN,NaN,41.0,NaN,4.0,NaN,NaN,https://www.lustat.ch/files_ftp/daten/covid/we...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18834,2022-04-12,NaN,SO,NaN,113171.0,NaN,23.0,3.0,NaN,NaN,362.0,https://corona.so.ch/bevoelkerung/daten/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18835,2022-04-12,NaN,TI,NaN,143931.0,NaN,111.0,3.0,NaN,NaN,1169.0,https://www4.ti.ch/fileadmin/DSS/DSP/UMC/malat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18836,2022-04-12,10:00,TG,NaN,117804.0,0.0,39.0,4.0,NaN,2510.0,648.0,https://www.tg.ch/news/fachstab-gesundheit-cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Remove Lichtenstein
data_local = data_local.drop(data_local[data_local["abbreviation_canton_and_fl"] == "FL"].index)

# Change data format according to the previous dataset
data_local["date"] = pd.to_datetime(data_local["date"])
data_local["Country or Region"] = "Switzerland"
data_local.rename(columns = {
    "date":"Date",
    "abbreviation_canton_and_fl":"Province or State",
    "ncumul_conf":"Confirmed",
    "ncumul_deceased":"Deaths",
    "ncumul_released":"Recovered",
    "current_quarantined":"Active"
}, inplace = True)

data_local = data_local[["Date", "Country or Region", "Province or State", "Confirmed", "Deaths", "Recovered", "Active"]]
data_local

,Date,Country or Region,Province or State,Confirmed,Deaths,Recovered,Active
0,2020-01-05,Switzerland,GE,4.0,0.0,NaN,NaN
1,2020-01-30,Switzerland,GE,5.0,0.0,NaN,NaN
2,2020-02-01,Switzerland,GE,5.0,0.0,NaN,0.0
3,2020-02-02,Switzerland,GE,5.0,0.0,NaN,0.0
4,2020-02-03,Switzerland,GE,5.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...
18833,2022-04-12,Switzerland,LU,NaN,NaN,NaN,NaN
18834,2022-04-12,Switzerland,SO,113171.0,362.0,NaN,NaN
18835,2022-04-12,Switzerland,TI,143931.0,1169.0,NaN,NaN
18836,2022-04-12,Switzerland,TG,117804.0,648.0,2510.0,NaN


In [22]:
# adding local data to global data
data = data.reset_index(drop = False)
data = data.append(data_local)
data = data.sort_values(["Date", "Country or Region", "Province or State", "Area"])
data = data.set_index(["Date", "Country or Region", "Province or State", "Area"])
data

Last Update  \
Date       Country or Region Province or State Area                       
2020-01-05 Switzerland       GE                NaN                  NaT   
2020-01-22 Antarctica        NaN               NaN  2020-01-22 17:00:00   
           China             Unknown           NaN  2020-01-22 17:00:00   
           Hong Kong         Hong Kong         NaN  2020-01-22 17:00:00   
           Japan             NaN               NaN  2020-01-22 17:00:00   
...                                                                 ...   
2022-04-12 Switzerland       BS                NaN                  NaT   
                             LU                NaN                  NaT   
                             SO                NaN                  NaT   
                             TG                NaN                  NaT   
                             TI                NaN                  NaT   

                                                     Latitude  Longitude  \
Date       Country or Region Province or State Area                        
2020-01-05 Switzerland       GE                NaN        NaN        NaN   
2020-01-22 Antarctica        NaN               NaN        NaN        NaN   
           China             Unknown           NaN        NaN        NaN   
           Hong Kong         Hong Kong         NaN        NaN        NaN   
           Japan             NaN               NaN        NaN        NaN   
...                                                       ...        ...   
2022-04-12 Switzerland       BS                NaN        NaN        NaN   
                             LU                NaN        NaN        NaN   
                             SO                NaN        NaN        NaN   
                             TG                NaN        NaN        NaN   
                             TI                NaN        NaN        NaN   

                                                     Confirmed  Deaths  \
Date       Country or Region Province or State Area                      
2020-01-05 Switzerland       GE                NaN         4.0     0.0   
2020-01-22 Antarctica        NaN               NaN         0.0     0.0   
           China             Unknown           NaN         NaN     NaN   
           Hong Kong         Hong Kong         NaN         NaN     NaN   
           Japan             NaN               NaN         2.0     NaN   
...                                                        ...     ...   
2022-04-12 Switzerland       BS                NaN     80710.0   282.0   
                             LU                NaN         NaN     NaN   
                             SO                NaN    113171.0   362.0   
                             TG                NaN    117804.0   648.0   
                             TI                NaN    143931.0  1169.0   

                                                     Recovered  Active  \
Date       Country or Region Province or State Area                      
2020-01-05 Switzerland       GE                NaN         NaN     NaN   
2020-01-22 Antarctica        NaN               NaN         0.0     NaN   
           China             Unknown           NaN         NaN     NaN   
           Hong Kong         Hong Kong         NaN         NaN     NaN   
           Japan             NaN               NaN         NaN     NaN   
...                                                        ...     ...   
2022-04-12 Switzerland       BS                NaN     79785.0     0.0   
                             LU                NaN         NaN     NaN   
                             SO                NaN         NaN     NaN   
                             TG                NaN      2510.0     NaN   
                             TI                NaN         NaN     NaN   

                                                     Incident Rate  \
Date       Country or Region Province or State Area                  
2020-01-05 Switzerland       